In [6]:
# import subprocess
# subprocess.run("ollama serve", shell=True)

In [7]:
# import ollama
# #Then everytime you want to chat
# response = ollama.chat(model='llama3.2', messages=[
#   {
#     'role': 'user',
#     'content': 'Why is the sky blue?',
#   },
# ])
# print(response['message']['content'])

In [1]:
import pandas as pd
import os
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAI, AzureOpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader

In [2]:
from dotenv import load_dotenv
load_dotenv() # read local .env file
OPENAI_API_BASE = os.getenv('OPENAI_API_BASE')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
OPENAI_API_BASE = os.getenv('OPENAI_API_BASE')
OPENAI_API_VERSION = os.getenv('OPENAI_API_VERSION')

In [2]:
import streamlit as st
# from streamlit_extras.add_vertical_space import add_vertical_space
from langchain_community.embeddings import OllamaEmbeddings
# from langchain_community.vectorstores import Chroma
# from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_community.chat_message_histories import StreamlitChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
# from langchain_openai import AzureOpenAI, AzureOpenAIEmbeddings
from langchain.chat_models import AzureChatOpenAI
from dotenv import load_dotenv
import os

In [3]:
embeddings = OllamaEmbeddings(model="nomic-embed-text", show_progress=False)


In [10]:
# embeddings = AzureOpenAIEmbeddings(
#     model="text-embedding-3-large",
#     deployment= 'genai-gpt-35-turbo',
#     # dimensions: Optional[int] = None, # Can specify dimensions with new text-embedding-3 models
#     azure_endpoint=OPENAI_API_BASE, #"https://<your-endpoint>.openai.azure.com/", If not provided, will read env variable AZURE_OPENAI_ENDPOINT
#     api_key=OPENAI_API_KEY, # Can provide an API key directly. If missing read env variable AZURE_OPENAI_API_KEY
#     openai_api_version=OPENAI_API_VERSION, # If not provided, will read env variable AZURE_OPENAI_API_VERSION
# )

In [4]:
import time

In [14]:
# # Function to process text data in chunks and add to the FAISS index
# def process_in_chunks(text_data, chunk_size, embeddings, index_name):
#     # Check if the index already exists
#     if os.path.exists(f"{index_name}.faiss") and os.path.exists(f"{index_name}.pkl"):
#         vectorstore = FAISS.load_local(index_name, embeddings)
#         print("Loaded existing FAISS index.")
#     else:
#         vectorstore = None
    
#     for i in range(0, len(text_data), chunk_size):
#         chunk = text_data[i:i + chunk_size]
#         try:
#             print(f"Processing chunk {i // chunk_size + 1}")
#             temp_vectorstore = FAISS.from_texts(chunk, embeddings)
            
#             if vectorstore:
#                 vectorstore.merge_from(temp_vectorstore)
#             else:
#                 vectorstore = temp_vectorstore
            
#             # Save the updated FAISS index after processing each chunk
#             vectorstore.save_local(index_name)
#             print(f"Chunk {i // chunk_size + 1} processed and index saved.")
            
#         except Exception as e:
#             print(f"Error processing chunk {i}: {e}")
        
#         time.sleep(1)  # Sleep to respect the rate limit
    
#     return vectorstore



In [9]:
# Initialize embeddings
embeddings = OllamaEmbeddings(model="nomic-embed-text", show_progress=False)

# Load documents
loader = DirectoryLoader("./input-data", glob="**/*.txt")
print("Directory loaded")
documents = loader.load()

# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""],
    chunk_size=1500,
    chunk_overlap=300,
    add_start_index=True,
)

# Split documents into chunks
texts = text_splitter.split_documents(documents)

Directory loaded


In [13]:
# Create FAISS index
vectorstore = FAISS.from_documents(texts, embeddings,)

print("FAISS index created successfully")

# Save the index locally
vectorstore.save_local("my_faiss_index")

print("FAISS index saved locally")

FAISS index created successfully
FAISS index saved locally


In [13]:
# Load the saved index
loaded_vectorstore = FAISS.load_local("my_faiss_index", embeddings, allow_dangerous_deserialization=True)

# Create retriever
retriever = loaded_vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)



In [5]:
load_dotenv() # read local .env file
OPENAI_API_TYPE = os.getenv('OPENAI_API_TYPE')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
OPENAI_API_BASE = os.getenv('OPENAI_API_BASE')
OPENAI_API_VERSION = os.getenv('OPENAI_API_VERSION')

In [6]:
llm = AzureChatOpenAI(openai_api_base=OPENAI_API_BASE,
                        openai_api_version=OPENAI_API_VERSION,
                        openai_api_key=OPENAI_API_KEY,
                        openai_api_type=OPENAI_API_TYPE,
                        #deployment_name = 'genai-gpt-4-32k',
                        #model_name = 'gpt-4-32k')
                        deployment_name = 'genai-gpt-35-turbo',
                        model_name = 'gpt-35-turbo')

C:\Users\750040697\AppData\Local\Temp\ipykernel_18016\3925094818.py:1: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  llm = AzureChatOpenAI(openai_api_base=OPENAI_API_BASE,


In [14]:
# # Create prompt template
template = """<bos><start_of_turn>user
You are a sales inquiry assistant specializing in rebates, market share, promotions, and payouts. Answer the question based only on the following context and provide a meaningful response. 
Write in full sentences with correct spelling and punctuation. Use bullet points or numbered lists when appropriate.
If the context doesn't contain the answer, politely state that you don't have the specific information and suggest contacting the sales department for more details.

CONTEXT: {context}

QUESTION: {question}

<end_of_turn>
<start_of_turn>model
ANSWER:"""
prompt = ChatPromptTemplate.from_template(template )

# # Modify the rag_chain to return the result instead of streaming
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [21]:
rag_chain.invoke("RfP affiliation change request")

'All affiliation change requests must be accompanied by documentation (such as an email) from the account requesting such. Please include account number, group to be affiliated with, and quarter to begin new affiliation. Alternatively, the account can send the request to RA VISUS SalesInquir@its.jnj.com directly. Ensure that they include SAP, date of affiliation change, and group to be affiliated with. An account must join, or have joined, the requested RfP group on or before the last day of the first month of the quarter to qualify for a rebate.'

In [37]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents import initialize_agent, AgentType, Tool
import pandas as pd

In [29]:
df = pd.read_csv(r'./car_dummy.csv')
df_columns = df.columns

In [43]:
df_agent = create_pandas_dataframe_agent(llm, df, verbose=True,allow_dangerous_code=True,max_iterations = 3)

# Create embeddings for the vector store
embeddings = OllamaEmbeddings(model="nomic-embed-text", show_progress=False)

# Load the saved index
loaded_vectorstore = FAISS.load_local("my_faiss_index", embeddings, allow_dangerous_deserialization=True)

# Create retriever
retriever = loaded_vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

# Create prompt template
template = """<bos><start_of_turn>user
You are a sales inquiry assistant specializing in rebates, market share, promotions, and payouts. Answer the question based on the following context, including retrieved documents and relevant employee data when applicable. 
Write in full sentences with correct spelling and punctuation. Use bullet points or numbered lists when appropriate.
If the context doesn't contain the specific information, politely state that you don't have the answer and suggest contacting the sales department for further details.

CONTEXT: {context}

DATAFRAME INFO: {dataframe_info} dataframe columns = {df_columns}

QUESTION: {question}

<end_of_turn>
<start_of_turn>model
ANSWER:"""
prompt = ChatPromptTemplate.from_template(template)

In [48]:
tools = [
    Tool(
        name="FAQ for sales Inquiries",
        func=retriever.get_relevant_documents,
        description="""
        Useful for when you need to answer questions about sales Inquiries.
        """
    ),
    Tool(
        name="Roster",
        func=df_agent.run,
        description=f"""
        Useful for when you need to answer questions about grouping, enrollment, location, eligibility, fell out, preferred, program, independent, roster, trackers, dashboard, joined, affiliation, fallen out, fallen off
        'rebate','market share','promotion','payout'  
        data stored in pandas dataframe 'df'. 
        'df' has the following columns: {df_columns}.
        """
    ),
]


In [49]:
# Initialize conversational memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [52]:
# Create custom agent prompt with friendly HR assistant context
agent_kwargs = {
    'prefix': f'''You are a sales inquiry assistant specializing in rebates, market share, promotions, and payouts. Answer the question based on the following context, including retrieved documents and relevant employee data when applicable. 
Write in full sentences with correct spelling and punctuation. Use bullet points or numbered lists when appropriate.
If the context doesn't contain the specific information, politely state that you don't have the answer and suggest contacting the sales department for further details. You have access to the following tools:'''
}

# Initialize the agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
    agent_kwargs=agent_kwargs,
    handle_parsing_errors=True,
    early_stopping_method="generate"
)

In [53]:
agent.run('Account is not found on the RfP Tracker')



> Entering new AgentExecutor chain...
I need to find out if the account exists in our system or if it is a new customer
Action: Roster
Action Input: Account name or ID

> Entering new AgentExecutor chain...
Thought: It's not clear what account name or ID is being referred to. Need more information.
Final Answer: N/A

> Finished chain.

Observation: N/A
Thought:I need to check if the account is a new customer or already exists in our system
Action: Roster
Action Input: Account name or ID

> Entering new AgentExecutor chain...
Thought: This question is unclear. It is not clear what account is being referred to.
Final Answer: N/A

> Finished chain.

Observation: N/A
Thought:I can't find any information about the account in our Roster, so it might be a new customer
Action: FAQ for sales Inquiries
Action Input: "How do I add a new customer to the RfP Tracker?"
Observation: [Document(metadata={'source': 'input-data\\inquiries.txt', 'start_index': 0}, page_content='RfP Alignments and Affili

'The account may be a new customer and the sales team should contact New Accounts to add them to the RfP Tracker.'

In [ ]:
# # Function to check if a question is related to structured data in the DataFrame
# def is_dataframe_question(query):
#     related_keywords = ['rebate', 'market share', 'promotion', 'payout']
#     return any(keyword in query.lower() for keyword in related_keywords)

# # Main chatbot logic
# def chatbot(query):
#     # Retrieve relevant documents from the vector store (always needed)
#     context = retriever.get_relevant_documents(query)
#     context_str = "\n".join([doc.page_content for doc in context])

#     # Check if the query is related to the pandas DataFrame
#     if is_dataframe_question(query):
#         # Query the pandas DataFrame agent
#         dataframe_info = df_agent.run(query)
#     else:
#         # If it's not related to the DataFrame, leave dataframe_info empty
#         dataframe_info = "No relevant structured data found."

#     # Create input for the LLM with both context and DataFrame info
#     final_input = template.format(context=context_str, dataframe_info=dataframe_info, question=query)

#     # Get response from the LLM
#     response = llm_for_pandas(final_input)
#     return response

In [33]:
# Function to check if a question is related to structured data in the DataFrame
# def is_dataframe_question(query):
#     related_keywords = ['rebate', 'market share', 'promotion', 'payout']
#     return any(keyword in query.lower() for keyword in related_keywords)

# Main chatbot logic
def chatbot(query):
    # Retrieve relevant documents from the vector store (always needed)
    context = retriever.get_relevant_documents(query)
    context_str = "\n".join([doc.page_content for doc in context])

    # # Check if the query is related to the pandas DataFrame
    # if is_dataframe_question(query):
        # Query the pandas DataFrame agent
    
    # Always run the pandas DataFrame agent query, with error handling
    try:
        dataframe_info = df_agent.run(query)
    except Exception as e:
        # Handle the error and provide a default response
        dataframe_info = f"No relevant structured data found in DataFrame"

    # Create input for the LLM with both context and DataFrame info
    final_input = template.format(context=context_str, dataframe_info=dataframe_info, df_columns= df_columns, question=query)

    # Get response from the LLM
    response = llm.invoke(final_input)
    return response

In [34]:
chatbot('Account is not found on the RfP Tracker')



> Entering new AgentExecutor chain...
Thought: We need more information to answer this question. What account are we looking for? Is it in a specific column of the dataframe?
Action: Check if a specific account is in the `RFP_Key` column of the dataframe
Action Input: `df[df['RFP_Key'] == 'account number']`Check if a specific account is in the `RFP_Key` column of the dataframe is not a valid tool, try one of [python_repl_ast].We need to find all unique values in the `RFP_Key` column to see if the account is present.
Action: Use the `unique()` method to get all unique values in the `RFP_Key` column
Action Input: `df['RFP_Key'].unique()`Use the `unique()` method to get all unique values in the `RFP_Key` column is not a valid tool, try one of [python_repl_ast].We need to search for the account in the `RFP_Key` column using a boolean expression.
Action: Use boolean indexing to check if the account is present in the `RFP_Key` column
Action Input: `df[df['RFP_Key'] == 'account number']`Use

AIMessage(content='An account will not appear on the RfP Tracker if it has $0 in sales for the current quarter or if it is not an active member of any RfP group. There are known issues when searching by account name in the RfP tracker. Please try searching by account number. If the issue persists, please contact the sales department for further assistance.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 1763, 'total_tokens': 1837, 'completion_tokens_details': None}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a65e5301-9e2d-4a94-a1a1-613a42a8989c-0')